In [1]:
# -*- coding: utf-8 -*-

import fasttext
from loadData import loadData
from nltk.corpus import reuters
import numpy as np

In [2]:
doc_id = reuters.fileids()
train_id = [d for d in doc_id if d.startswith('training/')]
test_id = [d for d in doc_id if d.startswith('test/')]

data, label = loadData()

In [24]:
train = data['train']
train_label = label['train']

folds = 10
proportion = len(train) / folds

for i in range(0, folds - 1):
    f = open('./crossValidation/cross_' + str(i), 'w+')
    for j in range(i * proportion, (i + 1) * proportion):
        for cat in train_label[j]:
            f.write('__label__' + cat + ' ')
        text = train[j]
        text.replace('\t', ' ').replace('\n', ' ')
        f.write(u' '.join(text.split()).encode('utf-8') + '\n');
    f.close()
    
f = open('./crossValidation/cross_' + str(folds - 1), 'w+')
for j in range((folds - 2) * proportion, len(train)):
    for cat in train_label[j]:
        f.write('__label__' + cat + ' ')
    text = train[j]
    text.replace('\t', ' ').replace('\n', ' ')
    f.write(u' '.join(text.split()).encode('utf-8') + '\n');
f.close()

In [88]:
models = {}

for cat in reuters.categories():
    clf = fasttext.load_model('./model/' + cat + '.model.bin', label_prefix = '__label__')
    models[cat] = clf

corrects = 0.0
for d in test_id:
    flag = True
    for cat in reuters.categories():
        clf = models[cat]
        label = clf.predict([reuters.raw(d)])[0][0]
        if (label == cat and cat in reuters.categories(d)) or (label != cat and cat not in reuters.categories(d)):
            flag = True
        else:
            flag = False
            break
    if flag:
        corrects += 1

print 'accuracy:', corrects / len(test_id)

0.366346472342


In [5]:
for cat in reuters.categories():
    train = open(cat + '_train', 'w+')
    test = open(cat + '_test', 'w+')
    
    for d in train_id:
        if cat in reuters.categories(d):
            train.write('__label__' + cat)
        else:
            train.write('__label__NOT' + cat)
        text = reuters.raw(d)
        text.replace('\t', ' ').replace('\n', ' ')
        train.write(' ' + u' '.join(text.split()).encode('utf-8') + '\n')    
    for d in test_id:
        if cat in reuters.categories(d):
            test.write('__label__' + cat)
        else:
            test.write('__label__NOT' + cat)
        text = reuters.raw(d)
        text.replace('\t', ' ').replace('\n', ' ')
        test.write(' ' + u' '.join(text.split()).encode('utf-8') + '\n')
    
    train.close()
    test.close()
    
    clf = fasttext.supervised(cat + '_train', cat + '.model', label_prefix = '__label__')
    result = clf.test(cat + '_test')

    print cat, ': ', result.precision

KeyboardInterrupt: 

In [ ]:
for cat in reuters.categories():
    test_pos = open('./testData' + cat + '_pos_test', 'w+')
    test_neg = open('./testData' + cat + '_neg_test', 'w+')

    for d in train_id:
        text = reuters.raw(d)
        text.replace('\t', ' ').replace('\n', ' ')
        if cat in reuters.categories(d):
            test_pos.write('__label__' + cat)
            test_pos.write(' ' + u' '.join(text.split()).encode('utf-8') + '\n')
        else:
            test_neg.write('__label__NOT' + cat)
            test_neg.write(' ' + u' '.join(text.split()).encode('utf-8') + '\n')
    
    test_pos.close()
    test_neg.close()

In [23]:
train = open('train_Reuters', 'w+')
test = open('test_Reuters', 'w+')

for d in train_id:
    for cat in reuters.categories(d):
        train.write('__label__' + cat + ' ')
    
    text = reuters.raw(d)
    text.replace('\t', ' ').replace('\n', ' ')
    train.write(u' '.join(text.split()).encode('utf-8') + '\n');

for d in test_id:
    for cat in reuters.categories(d):
        test.write('__label__' + cat + ' ')
    
    text = reuters.raw(d)
    text.replace('\t', ' ').replace('\n', ' ')
    test.write(' '.join(text.split()).encode('utf-8') + '\n');
    
train.close()
test.close()

clf = fasttext.supervised('train_Reuters', 'fastText.model', label_prefix = '__label__')

In [25]:
clf = fasttext.load_model('fastText.model.bin', label_prefix = '__label__')

test = data['test']
test_label = label['test']

In [38]:
score = clf.test('test_Reuters')
score.recall

0.499732905982906